## Открываем и изучаем файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head(10))
df.info()
df.describe()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [3]:
df.duplicated().sum()

0

Датасет без пропусков и дубликатов.

## Разбиваем данные на выборки

Разобьем на выборки.

In [4]:
df_train, df_1 = train_test_split(df, test_size=0.4, random_state=1488)
df_valid, df_test = train_test_split(df_1, test_size=0.5, random_state=1488)
train_features = df_train.drop(['is_ultra'], axis=1) 
train_target = df_train['is_ultra'] 
valid_features = df_valid.drop(['is_ultra'], axis=1) 
valid_target = df_valid['is_ultra'] 
test_features = df_test.drop(['is_ultra'], axis=1) 
test_target = df_test['is_ultra'] 

Итого у нас теперь три выборки:
- обучающая - df_train
- валидационная - df_valid
- тестовая - df_test

## Исследуем модели

Перед нами стоит задача классификации, потому будем работать с методами классификации

### Обучающее дерево

In [5]:
best_tree = 0
best_model_t = None
best_accuracy = 0
for i in range(1,100):
    model_t = DecisionTreeClassifier(random_state=1488, max_depth=i)
    model_t.fit(train_features, train_target)
    valid_predictions = model_t.predict(valid_features)
    valid_accuracy = accuracy_score(valid_target, valid_predictions)
    if valid_accuracy > best_accuracy:
        best_depth = i
        best_model_t = model_t
        best_accuracy = valid_accuracy
print(best_depth, best_model_t, best_accuracy, sep='\n')

6
DecisionTreeClassifier(max_depth=6, random_state=1488)
0.8118195956454122


Лучшая глубина 6, доля правильных ответов для валидационной воборки = 81,18%. Модель обучается быстро, хоть и обучилась 10 раз по разной глубине дерева. Проверим, какой результат будет у других моделей.

### Случайный лес


In [6]:
best_tree = 0
best_model_f = None
best_accuracy = 0
for i in range(1,20):
    for depth in range (1, 111):
        model_f = RandomForestClassifier(random_state=1488, n_estimators=i, max_depth=depth)
        model_f.fit(train_features, train_target)
        valid_predictions = model_f.predict(valid_features)
        valid_accuracy = accuracy_score(valid_target, valid_predictions)
        if valid_accuracy > best_accuracy:
            best_tree = i
            best_model_f = model_f
            best_accuracy = valid_accuracy
print(best_tree, best_model_f, best_accuracy, sep='\n')

7
RandomForestClassifier(max_depth=5, n_estimators=7, random_state=1488)
0.8180404354587869


Выбрали лучшее дерево из леса. Доля правильных ответов 81,8%. Это больше, чем при работе с одним деревом. Но процесс обучения намного дольше. 

### Логистическая регрессия


In [7]:
model_l = LogisticRegression(random_state=1488)
model_l.fit(train_features, train_target)
valid_predictions = model_l.predict(valid_features)
valid_accuracy = accuracy_score(valid_target, valid_predictions)
valid_accuracy

0.7620528771384136

Для логистической регрессии доля верных предсказаний 76,2%. Это меньше, чем у дерева или случайного леса.

## Проверяем модель на тестовой выборке

Обучающее дерево

In [8]:
ttest_predictions = best_model_t.predict(test_features)
ttest_accuracy = accuracy_score(test_target, ttest_predictions)
ttest_accuracy 

0.80248833592535

Случайный лес

In [9]:
ftest_predictions = best_model_f.predict(test_features)
ftest_accuracy = accuracy_score(test_target, ftest_predictions)
ftest_accuracy 

0.8055987558320373

Логистическая регрессия

In [10]:
ltest_predictions = model_l.predict(test_features)
ltest_accuracy = accuracy_score(test_target, ltest_predictions)
ltest_accuracy 

0.7620528771384136

При тестировнии на валидационных выборках, модели обучающего дерева и случайного леса показали почти одинаковый результат(разница 0,62%). Но при тестировании на тестовой выборке, модели показали разницу в 5,6%, в пользу обучающего дерева. Делаем вывод, что более правильно будет использовать модель обучающего дерева.

In [14]:
(ftest_accuracy - ttest_accuracy)*100

0.3110419906687367

## Проверяем модель на адекватность

In [13]:
import numpy as np
from sklearn.dummy import DummyClassifier
X = train_features
y = train_target
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
dummy_clf.predict(X)
dummy_clf.score(X, y)

0.6820539419087137

Проверил при помощи DummyClassifier. Обученая нами модель предсказывает лучше, чем он. Выходит, что она в целом работает и выдаёт результат лучше случайного.